In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import os
import numpy as np
import tensorflow as tf
import tqdm
import io
import imageio

from collections import deque
from skimage.transform import resize, rescale
from skimage.measure import compare_ssim as ssim
from scipy.cluster.vq import vq

import matplotlib.pyplot as plt

# Own libraries and modules
from compression import jpeg_helpers
from helpers import loading, plotting, utils, summaries, tf_helpers, dataset
from models import compression
from pyfse import pyfse

from training.compression import train_dcn, visualize_distribution, visualize_codebook

In [ ]:
training = {
    'n_epochs': 1501,
    'batch_size': 25,
    'patch_size': 128,
    'sample_dropout': False,
    'learning_rate': 1e-4,
    'learning_rate_reduction_schedule': 1000,
    'learning_rate_reduction_factor': 0.5,    
    'validation_schedule': 100,
    'convergence_threshold': 1e-4,
    'current_epoch': 0,
    'validation_is_training': True,
    'augmentation_probs': {
        'resize': 0.0,
        'flip_h': 0.5,
        'flip_v': 0.5,
        'gamma': 0.5
    }
}

## Load the dataset

In [ ]:
# Load data
data_directory = os.path.join('./data/raw/nip_training_data/Nikon D90/')

data = dataset.IPDataset(data_directory, n_images=5, v_images=5, load='xy', val_rgb_patch_size=128)

print('Sampling ref size: ', data.W, 'x', data.H)
for key in ['training', 'validation']:
    if 'x' in data[key]: print('{}/X : {}'.format(key, data[key]['x'].shape))
    if 'y' in data[key]: print('{}/Y : {}'.format(key, data[key]['y'].shape))
    if key == 'training':
        batch_x, batch_y = data.next_training_batch(0, 5, rgb_patch_size=128)
        print('training/batch', 'x', batch_x.shape if batch_x is not None else None, 'y', batch_y.shape if batch_y is not None else None)
    if key == 'validation':        
        batch_x, batch_y = data.next_validation_batch(0, 5)
        print('validation/batch', 'x', batch_x.shape if batch_x is not None else None, 'y', batch_y.shape if batch_y is not None else None)

In [ ]:
# Load data
data_directory = os.path.join('./data/compression/')

# data = dataset.IPDataset(data_directory, n_images=16000, v_images=800, load='y', val_rgb_patch_size=training['patch_size'])
data = dataset.IPDataset(data_directory, n_images=450, v_images=50, load='y', val_rgb_patch_size=training['patch_size'])
# data = dataset.IPDataset(data_directory, n_images=1000, v_images=50, load='y', val_rgb_patch_size=training['patch_size'])

for key in ['training', 'validation']:
    print('{} : {}'.format(key, data[key]['y'].shape))

In [ ]:
# Show sample batch from the database
if 'batch_id' not in globals():
    batch_id = 0
    n_batches = data['training']['y'].shape[0] // training['batch_size']
    
batch_id = (batch_id + 1) % n_batches

print('Batch id: {} ({})'.format(batch_id, training['batch_size']))

batch_x = data.next_training_batch(batch_id, training['batch_size'], training['patch_size'])
fig = plotting.imsc(batch_x[:8], ncols=8, figwidth=25)

batch_x = data.next_validation_batch(batch_id, training['batch_size'])
fig = plotting.imsc(batch_x[:8], ncols=8, figwidth=25)

## Create DCN instance

In [ ]:
graph = tf.Graph()
sess = tf.Session(graph=graph)

# dcn = compression.AutoencoderDCN(sess, graph, patch_size=training['patch_size'], n_latent=0, n_filters=8, n_fscale=2, n_layers=2, res_layers=1,
#             rounding='soft', dropout=False, use_batchnorm=True, train_codebook=False, latent_bpf=5, scale_latent=False, entropy_weight=None)

dcn = compression.WaveOne(sess, graph, patch_size=training['patch_size'], rounding='soft', latent_bpf=8, scale_latent=True)

# dcn = twitter.TwitterDCN(sess, graph, patch_size=training['patch_size'], rounding='soft', latent_bpf=1, scale_latent=True)

print(dcn.summary())
print(dcn.model_code)
print('Compression stats:', dcn.compression_stats(n_latent_bytes=0.5))

In [ ]:
from helpers import tf_helpers
tf_helpers.show_graph(dcn.graph.as_graph_def())

## Training

In [ ]:
train_dcn({'dcn': dcn}, training, data, './data/raw/compression_playground/')

### Save model

In [ ]:
# Batch norm stats
prebn = dcn.sess.run(dcn.pre_bn, feed_dict={dcn.x: batch_x})
bM = np.mean(prebn, axis=(0,1,2))
bV = np.var(prebn, axis=(0,1,2))
pM = dcn.sess.run(dcn.graph.get_tensor_by_name('autoencoderdcn/encoder/bn_0/moving_mean:0'))
pV = dcn.sess.run(dcn.graph.get_tensor_by_name('autoencoderdcn/encoder/bn_0/moving_variance:0'))
print(prebn.shape)
print(bM.shape)
print(bV.shape)
print(pM.shape)
print(pV.shape)

fig, axes = plotting.sub(2)
axes[0].plot(bM, pM, 'o')
axes[1].plot(bV, pV, 'o')

In [ ]:
n_elements = 8
batch_y_bstats = dcn.process(batch_x[:n_elements], is_training=True)
batch_y_pstats = dcn.process(batch_x[:n_elements], is_training=False)
fig = plotting.imsc(batch_x[:n_elements], ncols=np.max((8, n_elements)), figwidth=25, titles='Input')
fig = plotting.imsc(batch_y_bstats[:n_elements], ncols=np.max((8, n_elements)), figwidth=25, titles='is_training=True (bStats)')
fig = plotting.imsc(batch_y_pstats[:n_elements], ncols=np.max((8, n_elements)), figwidth=25, titles='is_training=False (pStats)')

In [ ]:
import matplotlib.pyplot as plt
from helpers import utils

fig = plt.figure(figsize=(20, 4))
ax = fig.gca()
ax.plot(utils.ma_conv(perf['loss']['training'], n=11))
# ax.plot(np.arange(0, len(loss['training']), sampling_rate), utils.ma_conv(loss['validation'], n=3))
ax.plot(np.arange(0, len(perf['loss']['training']), sampling_rate), perf['loss']['validation'], '-o', alpha=0.3)
ax.plot(perf['loss']['training'], '.', alpha=0.1)
ax.legend(['train', 'valid'], loc='upper right')
# ax.set_yscale('log')

In [ ]:
dcn.load_model(os.path.join('./data/raw/compression/', dcn.short_name()))

In [ ]:
# Show a sample and a reconstruction of the current batch
batch_y = dcn.process(batch_x)
f = plotting.imsc(batch_x[0:8], ncols=8, figwidth=20)
f = plotting.imsc(batch_y[0:8], ncols=8, figwidth=20)

## Explore & Understand the Latent Representation

In [ ]:
fig = plotting.imsc(visualize_codebook(dcn))

In [ ]:
np.histogram(batch_z)

In [ ]:
# Show the distribution of the latent features

graph = tf.Graph()
sess = tf.Session(graph=graph)

dcn = compression.AutoencoderDCN(sess, graph, patch_size=training['patch_size'], n_latent=0, n_filters=8, n_fscale=2, n_layers=2, r_layers=0, 
            rounding='soft', dropout=False, use_batchnorm=True, train_codebook=False, latent_bpf=5, scale_latent=False, entropy_weight=None)

dcn.init()

fig = plotting.imsc(visualize_distribution(dcn, data), figwidth=15)

with dcn.graph.as_default():
    histogram = dcn.sess.run(dcn.histogram, feed_dict={dcn.x: batch_x, dcn.is_training: True})
    entropy = dcn.sess.run(- tf.reduce_sum(dcn.histogram * tf.log(dcn.histogram)) / 0.6931, feed_dict={dcn.x: batch_x, dcn.is_training: True})

plt.plot(histogram)
print('TF et entropy', entropy)

print('NP est entropy', - np.sum(histogram * np.log2(histogram)))

bin_centers = np.arange(-32, 32, 0.1)
bin_boundaries = np.convolve(bin_centers, [0.5, 0.5], mode='valid')
bin_centers = bin_centers[1:-1]

batch_z = dcn.compress(batch_x, is_training=True)
# Compute frequencies using a histogram
# freq = np.histogram(batch_z[:], bins=bin_boundaries, normed=False)[0]
hist = np.histogram(batch_z[:], bins=bin_boundaries, density=True)[0]
hist = hist.clip(1e-9)
probs = hist / np.sum(hist)

entropy_emp = np.sum(- probs * np.log2(probs))
print('Empirical entropy', entropy_emp)

In [ ]:
dcn.init()

In [ ]:
# batch_z = dcn.sess.run(dcn.latent_pre, feed_dict={dcn.x: batch_x, dcn.is_training: True})
# op = dcn.graph.get_operation_by_name('autoencoderdcn/encoder/conv_0/LeakyRelu')
tn = dcn.graph.get_tensor_by_name('autoencoderdcn/encoder/conv_0/weights:0')
print(tn.name)
batch_z = dcn.sess.run(tn, feed_dict={dcn.x: batch_x, dcn.is_training: True})

# batch_z = dcn.compress(batch_x)
print(batch_z.shape)
print(batch_z)

In [ ]:
# Show the distribution of the latent features

sample_batch_size = 50
batch_x = data.next_validation_batch(0, sample_batch_size)

# See latent distribution
batch_z = dcn.compress(batch_x)
# batch_z = dcn.compress_soft(batch_x)
batch_z = batch_z.reshape((sample_batch_size, -1)).T
print(batch_z.shape)

bin_centers = np.arange(-32, 32, 0.1)
bin_boundaries = np.convolve(bin_centers, [0.5, 0.5], mode='valid')
bin_centers = bin_centers[1:-1]

hist = np.histogram(batch_z[:], bins=bin_boundaries, normed=True)[0]

# for bin, value in zip(bin_centers, hist):
#     print('{:.3f}'.format(bin), '->', value)   

fig = plt.figure(figsize=(20, 4))
ax = fig.gca()
ax.bar(bin_centers, hist, width=bin_centers[1] - bin_centers[0], color='r')
ax.set_title('Histogram of quantized coefficients')

In [ ]:
# Show an example of soft quantization & compute entropy

# Prepare the quantization codebook
q_min = np.percentile(batch_z.astype(np.int32), 0.1)
q_max = np.percentile(batch_z.astype(np.int32), 100 - 0.1)

bin_boundaries = np.arange(q_min + 0.5, q_max + 0.5, 1)
bin_centers = np.convolve(bin_boundaries, [0.5, 0.5], mode='valid')

print('Bin centers ({}): {}'.format(len(bin_centers), bin_centers.tolist()))

batch_ex = batch_z.T[0]
value = batch_ex.reshape((4096, 1))

# Compute soft quantization
sigma = 0.1
weights = np.exp(-sigma * np.power(value - bin_centers.reshape(1, len(bin_centers)), 2))
weights = weights / weights.sum(axis=1, keepdims=True)
soft = np.sum(weights * bin_centers, axis=1)

# Compute frequencies using unique
indices, _ = vq(batch_z.reshape((-1, )), bin_centers)
unique, counts = np.unique(bin_centers[indices], return_counts=True)
counts = counts / counts.sum()

# Compute soft histogram
histogram = np.mean(weights, axis=0).clip(1e-6)
histogram = histogram / np.sum(histogram)

entropy = - np.sum(counts * np.log2(counts))
entropy_estimate = - np.sum(histogram * np.log2(histogram))

# Print stats
feature_id = 1
print('Feature', feature_id)
print('Value', value[feature_id])
print('Hard Quantization', np.round(value[feature_id]))
print('Soft quantization', np.sum(weights[feature_id] * bin_centers))

# Plot results
fig, axes = plotting.sub(2, figwidth=16, figheight=4)

axes[0].plot(bin_centers, weights[feature_id])
axes[0].set_title('Soft quantization of a selected feature {:.3f} / {:.3f}'.format(batch_ex[feature_id], np.sum(soft[feature_id])))

axes[1].plot(unique, counts)
axes[1].plot(bin_centers, histogram)
axes[1].legend(['unique', 'soft'])
axes[1].set_title('Actual vs. soft estimated distributions - H={:.2f} vs. {:.2f}'.format(entropy_estimate, entropy))

## Entropy Coding

In [ ]:
latent_means = np.mean(batch_z,axis=1)
fig, axes = plotting.sub(2, figwidth=12)
axes[0].plot(latent_means)
axes[0].set_title('Mean values for all latent variables')
axes[1].hist(latent_means, bins=50, normed=True)
axes[1].set_title('Histogram of mean values for all latent variables')

In [ ]:
batch_x = data.next_validation_batch(0, 400)

batch_z = dcn.compress(batch_x)
batch_z = batch_z.reshape((sample_batch_size, -1)).T
print(batch_z.shape)

n_bins = 64
bin_boundaries = np.linspace(-16, 16, n_bins+1)
bin_centers = np.convolve(bin_boundaries, [0.5, 0.5], mode='valid')

print(bin_centers)

distribution = np.zeros((len(batch_z), n_bins))
for i in range(len(batch_z)):
    distribution[i, :] = np.histogram(batch_z[i], bins=bin_boundaries, normed=True)[0]

vis = []
for i in range(len(distribution) // 512):
    vis.append(distribution[i*512:(i+1)*512, :])
    

thumbs = plotting.thumbnails(vis, n_cols=len(vis))
    
fig = plotting.imsc(thumbs, 'A', figwidth=20)

In [ ]:
n_bins = 64
bin_boundaries = np.linspace(-256, 256, n_bins+1)
bin_centers = np.convolve(bin_boundaries, [0.5, 0.5], mode='valid')

i = 58
fig = plt.figure(figsize=(16, 4))
fig.gca().fill_between(bin_centers, 0, np.histogram(batch_z[i], bins=bin_boundaries, normed=True)[0], alpha=0.1)
print(distribution[i, :])
print(batch_z[i])

In [ ]:
fig = plt.figure(figsize=(16, 4))
for i in range(len(batch_z)):
#     fig.gca().plot(bin_centers, distribution[i, :])
    fig.gca().fill_between(bin_centers, 0, distribution[i, :], alpha=0.1)

In [ ]:
for op in dcn.sess.graph.get_operations():
    print(op.name)

## Experiments with Entropy Coding

In [ ]:
# Load a pre-trained model
from test_dcn import restore_model
from dahuffman import HuffmanCodec

dirname = './data/raw/compression_entropy/{}/'
dcn_model = 'TwitterDCN-16384D/16x16x64-r:soft-Q-8.0bpf-S+-H+250.00'
dcn_model = 'TwitterDCN-16384D/16x16x64-r:soft-codebook-Q-4.0bpf-S+-H+100.00'


dcn = restore_model(dirname.format(dcn_model), patch_size=256)

print('', dcn._h.to_dict())

In [ ]:
## Load an example image
from helpers import coreutils

dirname = './data/clic256/'
filenames = coreutils.listdir(dirname, '.*\.png$')
# filenames = ['alejandro-escamilla-6.png']

image = imageio.imread(os.path.join(dirname, filenames[1])).astype(np.float32) / (2**8 - 1)
# image = imageio.imread('./data/clic512/casey-fyfe-999.png').astype(np.float32) / (2**8 - 1)
image = resize(image, (dcn.patch_size, dcn.patch_size))
batch_x = np.expand_dims(image, axis=0)
# batch_x = utils.slidingwindow(image, 128)
# batch_x = batch_x[6:7]
fig = plotting.imsc(batch_x, figwidth=4)

In [ ]:
# Test a layer-wise AFI codec
from compression import afi

compressed_image = afi.afi_compress(dcn, batch_x, verbose=False)
batch_r = afi.afi_decompress(dcn, compressed_image, verbose=False)

print('Compressed AFI: {:,} bytes'.format(len(compressed_image)))
ssim_value = ssim(batch_x.squeeze(), batch_r.squeeze(), multichannel=True, data_range=1)

fig = plotting.imsc((batch_x, batch_r), ['input image', 'decompressed image ssim={:.2f}'.format(ssim_value)], figwidth=10)

In [ ]:
# Compare global vs. per-layer encoding
# TODO Entropy is computed incorrectly for non-binary cases

batch_z = dcn.compress(batch_x)
code_book = dcn.get_codebook()

# Global
indices, _ = vq(batch_z.reshape((-1)), code_book)
coded_image = pyfse.easy_compress(bytes(indices.astype(np.uint8)))

# Per-feature
entropies = []
indices_pf = []
lengths_pf = []
n_layers = batch_z.shape[-1]
for n in range(n_layers):
    indices, _ = vq(batch_z[:, :, :, n].reshape((-1)), code_book)    
    coded_layer = pyfse.easy_compress(bytes(indices.astype(np.uint8)))
    indices_pf.append(indices)
    lengths_pf.append(len(coded_layer))
    entropies.append(utils.entropy(batch_z[:, :, :, n], code_book))

expected_layer_size = dcn.latent_bpf * np.prod(batch_z.shape[1:-1]) / 8

print('Latent    : {} = {:,}'.format(dcn.latent_shape, np.prod(dcn.latent_shape)))
print('Naive     : {:,.0f} bytes'.format(dcn.latent_bpf * dcn.n_latent / 8))
print('Global    : {:,} bytes'.format(len(coded_image)))
print('Per-layer : {:,} bytes ({:.1f}%)'.format(sum(lengths_pf), 100 * sum(lengths_pf) / len(coded_image)))
print('Expected layer size: {:,.0f} bytes'.format(expected_layer_size))
print('\nActual vs. expected layer code [bytes] :')
print(np.array(lengths_pf) - expected_layer_size)
print('\nLayer entropy (av {:.2f}):'.format(np.mean(entropies)))
print(np.array(entropies).round(2))

In [ ]:
# Visualize feature maps
batch_f = np.expand_dims(np.moveaxis(batch_z[:,:,:,:].squeeze(), 2, 0), axis=3)
fig = plotting.imsc(batch_f, titles=['{}: H={:.2f} / {}'.format(x, e, l) for x, (e, l) in enumerate(zip(entropies, lengths_pf))], figwidth=35, ncols=16)

In [ ]:
## Compare the DCN and JPEG

batch_z = dcn.compress(batch_x, is_training=False)
batch_d = dcn.process(batch_x, is_training=False)

if dcn._h.rounding == 'identity':
#     code_book = dcn.get_codebook(bpf=8)
    code_book = np.array(code_books[16]).reshape((-1,))
#     code_book = np.linspace(-2, 2, 256)
else:
    code_book = dcn.get_codebook(bpf=8)
    code_book_edges = np.convolve(code_book, [0.5, 0.5], mode='valid')

indices, _ = vq(batch_z.reshape((-1, )), code_book)
print('Codebook size: {} // {}'.format(len(code_book), code_book.tolist()))
counts = utils.qhist(batch_z, code_book)

counts = counts.clip(min=1)
probs = counts / counts.sum()
entropy = - np.sum(probs * np.log2(probs))

# Plot distributions --------------------------------------------------------------------------------------
density = True

fig, axes = plotting.sub(2)
fig.set_size_inches(20, 5)

for r in range(2):
    axes[r].hist(batch_z.reshape((-1,1)), bins=utils.bin_egdes(code_book), density=density, alpha=0.5)
    axes[r].hist(batch_z.reshape((-1,1)), bins=500, density=density, alpha=0.5)
    axes[r].plot(code_book, np.histogram(batch_z.reshape((-1,1)), bins=utils.bin_egdes(code_book), density=density)[0], 'k-')
    axes[r].plot(code_book, utils.qhist(batch_z, code_book, density=density), 'o--')
    axes[r].legend(['hist/codebook', 'hist/uniform', 'np.hist/codebook', 'utils.hist/codebook'])
    if r == 1:
        axes[r].set_xlim([np.percentile(batch_z, 1), np.percentile(batch_z, 99)])
# ---------------------------------------------------------------------------------------------------------

In [ ]:
# Vector quantization
indices, distortion = vq(batch_z.reshape((-1)), code_book)
batch_q = code_book[indices]
batch_y = dcn.decompress(batch_q.reshape(dcn.latent_shape))

ssim_value = ssim(batch_x[0], batch_y[0], multichannel=True, data_range=1)
ssim_value_direct = ssim(batch_x[0], batch_d[0], multichannel=True, data_range=1)

# ssim_value = msssim(batch_x[0], batch_y[0])
# ssim_value_direct = msssim(batch_x[0], batch_d[0])

# Naive FSE compression of the entire latent repr.
coded_fse = pyfse.easy_compress(bytes(indices.astype(np.uint8)))

# Per-layer compression
layer_coded_fse = afi.afi_compress(dcn, batch_x)

print('DCN             : {}'.format(dcn.model_code))
print('Pixels          : {}x{} = {:,}'.format(image.shape[0], image.shape[1], np.prod(image.shape[:2])))
print('Bitmap          : {:,} bytes'.format(np.prod(image.shape)))
print('Batch size      : {:,} elements'.format(np.prod(batch_x.shape)))
print('Code-book size  : {} elements'.format(len(code_book)))
print('Entropy         : {:.2f} bits per symbol'.format(entropy))
print('Latent size     : {:,}'.format(np.prod(batch_z.shape)))
print('PPF Naive       : {:,.0f} --> {:,.0f} bytes [{} bits per element]'.format(
    np.prod(batch_z.shape) * np.log2(len(code_book)) / 8,
    np.prod(batch_z.shape) * np.ceil(np.log2(len(code_book))) / 8,
    np.ceil(np.log2(len(code_book)))
    ))
print('PPF Theoretical : {:,.0f} bytes ({:.2f} bpp)'.format(np.prod(batch_z.shape) * entropy / 8, 
                                                            np.prod(batch_z.shape) * entropy / np.prod(batch_x.shape[1:3])))
print('FSE Coded       : {:,.0f} bytes ({:.2f} bpp)'.format(len(coded_fse), 
                                                            8 * len(coded_fse) / np.prod(batch_x.shape[1:3])))
print('FSE Layer-Coded : {:,.0f} bytes ({:.2f} bpp) --> ssim: {:.3f}'.format(len(layer_coded_fse), 
                                                            8 * len(layer_coded_fse) / np.prod(batch_x.shape[1:3]), ssim_value))

# Encode JPEG
try:
    jpeg_quality = jpeg_helpers.match_ssim(batch_x[0], ssim_value, subsampling='4:4:4')
except:
    jpeg_quality = 95

batch_j, jpeg_bytes = jpeg_helpers.compress_batch(batch_x, jpeg_quality, effective=True, subsampling='4:4:4')
ssim_value_jpeg = ssim(batch_x[0], batch_j[0], multichannel=True, data_range=1)
jpeg_bytes = jpeg_bytes[0]

print('JPEG (Q={:2d})     : {:,} bytes ({:0.2f} bpp) --> ssim: {:.3f}'.format(
    jpeg_quality, jpeg_bytes, 8 * jpeg_bytes / np.prod(batch_j.shape[1:3]), ssim_value_jpeg ))

# If processing a batch, choose an example to show
example_id = 0
batch_all = np.concatenate((batch_x[example_id:example_id+1], batch_d[example_id:example_id+1], batch_y[example_id:example_id+1], batch_j[example_id:example_id+1]), axis=0)

plot_titles = ['Original', 'DCN direct ({:.2f})'.format(ssim_value_direct), 'DCN codec ({:.3f})'.format(ssim_value), 'JPEG Q={} ({:.3f})'.format(jpeg_quality, ssim_value_jpeg)]
f = plotting.imsc(batch_all, titles=plot_titles, ncols=len(batch_all), figwidth=25)

## Other Experiments

In [ ]:
counts = utils.qhist(batch_z, code_books[bins])
cc = plt.hist(batch_z.reshape((-1,1)), bins=50)

cc = np.histogram(batch_z.reshape((-1,1)))[0]

print(sum(cc))
print(sum(counts))

In [ ]:
print(code_book)
np.trapz(np.ones_like(code_book), code_book)

In [ ]:
bins = 32
plt.hist(batch_z.reshape((-1,1)), bins=bins, density=True)
plt.plot(code_books[bins], utils.qhist(batch_z, code_books[bins], density=True))

In [ ]:
code_book = sorted(quantization.generate_codebook(
    np.abs(batch_z.reshape((-1,1))[::10]), 
    size_codebook=bins
)[0])

In [ ]:

np.sort(np.concatenate((-np.array(code_book), np.zeros((1,1)), np.array(code_book)), axis=0).reshape(-1,))

In [ ]:
plt.plot(code_book, np.ones_like(code_book), 'x')

In [ ]:
from helpers import quantization

In [ ]:
add_zero = False

code_books = {}
for bins in [4, 8, 16, 32, 64, 128, 256]:
    print(bins)
    
    code_book = sorted(quantization.generate_codebook(
        np.abs(batch_z.reshape((-1,1))[::10]), 
        size_codebook=bins//2
    )[0])
    
    if add_zero:
        code_books[bins] = np.sort(np.concatenate((-np.array(code_book[:-1]), np.zeros((1,1)), np.array(code_book)), axis=0).reshape(-1,))
    else:
        code_books[bins] = np.sort(np.concatenate((-np.array(code_book), np.array(code_book)), axis=0).reshape(-1,))
        
    print(len(code_books[bins]))

In [ ]:
for bins in [4, 8, 16, 32, 64, 128, 256]:
    plt.plot(code_books[bins], 'o-')

In [ ]:
print('{')
for k, v in code_books.items():
    pp = ', '.join(['{:.6f}'.format(x) for x in v])
    print('  {}: np.array([{}]),'.format(k, pp))
print('}')    

In [ ]:
bins = 32
plt.figure(figsize=(20,2))
plt.plot(code_books[bins], np.ones_like(code_books[bins]), 'x')

In [ ]:
plt.plot(
    batch_z.reshape((-1,1)),
    batch_q.reshape((-1,1)),
    '.'
)

In [ ]:
from compression import afi

out_a, out_b = afi.dcn_compare(dcn, batch_x)

diff = np.abs(out_a - out_b)
diff /= diff.max()

fig = plotting.imsc((out_a, out_b, diff), ncols=3, figwidth=30)

In [ ]:
batch_p = data.next_validation_batch(0, 5)

batch_g = utils.batch_gamma(batch_p)

fig = plotting.imsc(batch_p, ncols=5, figwidth=20)
fig = plotting.imsc(batch_g, ncols=5, figwidth=20)

## How to package everything into a file format

In [ ]:
for op in dcn.graph.get_operations():
    print(op.name, op.outputs[0].shape)

In [ ]:
tf.constant(np.arange(-10, 10 + 1), shape=(1, 21), dtype=tf.float32).shape